In [9]:
from pychunkedgraph.graph import ChunkedGraph, attributes
import numpy as np
import collections
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
import matplotlib.dates as mdates
import numpy as np
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg


In [14]:
dataset = "fish_v241003_test"
cg = ChunkedGraph(graph_id=dataset)

### Get all operations

In [15]:
max_op_id = cg.client.get_max_operation_id()
max_op_id

4983

In [5]:
url = "https://pcgv3local.brain-wire-test.org"
global_url = "https://global.brain-wire-test.org/"
datastack = "fish1_full"
dataset = "fish_v241003"
resolution = [16, 16, 30]

In [10]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)
cggraph = cg.ChunkedGraphClient(server_address=url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))
cv = client.info.segmentation_cloudvolume()

In [13]:
cggraph.get_operation_details(list(range(22530050-100, 22530050)))

{'22529953': {'added_edges': [[83673249472315839, 83532649557131295]],
  'operation_exception': '',
  'operation_status': 0,
  'operation_ts': '2025-01-07 14:34:10.173000+00:00',
  'roots': [864691128661129585],
  'sink_coords': [[41844, 18457, 4608]],
  'source_coords': [[42249, 18027, 4392]],
  'timestamp': '2025-01-07 14:34:10.173000+00:00',
  'user': '31'},
 '22529958': {'added_edges': [[80718447331246344, 80577366648160613]],
  'operation_exception': '',
  'operation_status': 0,
  'operation_ts': '2025-01-07 14:34:18.559000+00:00',
  'roots': [864691128667069921],
  'sink_coords': [[31206, 19223, 1157]],
  'source_coords': [[31743, 20695, 423]],
  'timestamp': '2025-01-07 14:34:18.559000+00:00',
  'user': '31'},
 '22529963': {'added_edges': [[79802899823067255, 79803106317043209]],
  'operation_exception': '',
  'operation_status': 0,
  'operation_ts': '2025-01-07 14:34:24.333000+00:00',
  'roots': [864691128658209282],
  'sink_coords': [[28372, 18501, 5090]],
  'source_coords': [

In [16]:
max_op_id = cg.client.get_max_operation_id()
# max_op_id = 5000000
log_rows = {}
num_sections = max(1, max_op_id // 100000)
for id_batch in np.array_split(np.arange(0, max_op_id), num_sections):
    %time log_rows.update(cg.client.read_log_entries(operation_ids=id_batch))

CPU times: user 796 ms, sys: 22.2 ms, total: 818 ms
Wall time: 1.27 s


In [17]:
log_data = collections.defaultdict(list)

failed_operations = 0
for operation_id, log_row in log_rows.items():
    if log_row.get(attributes.OperationLogs.Status, 0) != 0:
        # continue
        failed_operations += 1
        
    # log_data["date"].append(log_row["timestamp"])
    # log_data["timestamp"].append(log_row["timestamp"].timestamp())
    # log_data["user_id"].append(log_row[attributes.OperationLogs.UserID])
    
    # if log_row[attributes.OperationLogs.SinkCoordinate][0][0] > 1e9:
    #     sinks = np.frombuffer(log_row[attributes.OperationLogs.SinkCoordinate]).reshape(-1, 3)
    #     sources = np.frombuffer(log_row[attributes.OperationLogs.SourceCoordinate]).reshape(-1, 3)
    # else:
    #     sinks = log_row[attributes.OperationLogs.SinkCoordinate]
    #     sources = log_row[attributes.OperationLogs.SourceCoordinate]
    # mean_coord = (np.mean(sinks, axis=0) + np.mean(sources, axis=0)) / 2
    # log_data["coord_x"].append(mean_coord[0])
    # log_data["coord_y"].append(mean_coord[1])
    # log_data["coord_z"].append(mean_coord[2])
    # log_data["operation_id"].append(operation_id)

In [18]:
failed_operations

2

### Plot results

In [8]:
# Sort the dates to ensure the cumulative sum is calculated in order
# dates = sorted(log_data['date'])
dates = [d.replace(tzinfo=None) for d in sorted(log_data['date'])]

# Initialize cumulative count
cumulative_counts = []
cumulative_sum = 0

# Calculate cumulative sum of edits over time
for _ in dates:
    cumulative_sum += 1
    cumulative_counts.append(cumulative_sum)

def add_months(source_date, months):
    # Calculate the new month and year considering year overflow
    month = source_date.month - 1 + months
    year = source_date.year + month // 12
    month = month % 12 + 1
    day = min(source_date.day, [31, 29 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][month - 1])
    return datetime(year, month, day, source_date.hour, source_date.minute, source_date.second, source_date.microsecond)

range_dates = []
range_counts = []

next_marker_date = dates[0]  # Start from the first date
print(dates, next_marker_date)
for i, date in enumerate(dates):
    if date >= next_marker_date:  # If current date is on or after the next marker date
        range_dates.append(date)
        range_counts.append(cumulative_counts[i])
        next_marker_date = add_months(next_marker_date, 3)

# Plot the line for all data
plt.figure(figsize=(10, 6))
plt.plot(dates, cumulative_counts, linestyle='-', color='b')

# Add circle markers at exact intervals
plt.plot(range_dates, range_counts, marker='o', linestyle='None', color='r')

# Formatting the date on the x-axis
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format the x-axis labels as Year-Month
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Set the locator to intervals

plt.gcf().autofmt_xdate()  # Auto format the x-axis for better readability
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.ylabel('Cumulative Edit Count')
plt.title('{}: Cumulative Number of Edits Over Time'.format(dataset))
# plt.grid(True)
plt.legend()
plt.show()

[datetime.datetime(2022, 10, 1, 17, 23, 11, 400000), datetime.datetime(2022, 10, 1, 17, 50, 59, 205000), datetime.datetime(2022, 10, 1, 17, 51, 32, 424000), datetime.datetime(2022, 10, 1, 17, 51, 49, 209000), datetime.datetime(2022, 10, 1, 17, 52, 2, 206000), datetime.datetime(2022, 10, 1, 20, 28, 35, 567000), datetime.datetime(2022, 10, 1, 20, 28, 50, 647000), datetime.datetime(2022, 10, 1, 20, 29, 23, 544000), datetime.datetime(2022, 10, 1, 20, 31, 6, 840000), datetime.datetime(2022, 10, 2, 17, 56, 8, 148000), datetime.datetime(2022, 10, 2, 17, 56, 32, 728000), datetime.datetime(2022, 10, 2, 17, 57, 37, 372000), datetime.datetime(2022, 10, 2, 17, 58, 22, 985000), datetime.datetime(2022, 10, 2, 17, 58, 35, 135000), datetime.datetime(2022, 10, 2, 17, 58, 43, 278000), datetime.datetime(2022, 10, 2, 17, 59, 24, 771000), datetime.datetime(2022, 10, 2, 17, 59, 39, 769000), datetime.datetime(2022, 10, 2, 17, 59, 58, 949000), datetime.datetime(2022, 10, 2, 18, 0, 48, 476000), datetime.dateti

NameError: name 'datetime' is not defined